In [1]:
# Module Importations
import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

# Print versioning information
print('keras version =', keras.__version__)
print('numpy version =', np.__version__)  
print('sklearn version =', sklearn.__version__)
print('tensorflow version =', tf.__version__)

keras version = 2.4.0
numpy version = 1.19.4
sklearn version = 0.22.2.post1
tensorflow version = 2.4.1


In [2]:
# Custom Module Imports
from Source.data import load_data
from Source.data import split_data
from Source.models import model_evaluation
from Source.models import keras_helpers
from Source.models import tensorboard_helpers

[keras_helpers]Tensorflow version: 2.4.1
[keras_helpers]keras version = 2.4.0


In [3]:
# Constants
TRAIN_MODELS = True

In [4]:
# Load dataset
df_plant1 = load_data.load_pickled_data("df_plant1_feat_eng.pkl")
df_plant2 = load_data.load_pickled_data("df_plant2_feat_eng.pkl")

Loading pickled dataframe started ...
Loading pickled dataframe complete ...
Loading pickled dataframe started ...
Loading pickled dataframe complete ...


In [5]:
df_plant1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71808 entries, 0 to 71806
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DATE_TIME        71808 non-null  datetime64[ns]
 1   PLANT_ID         71808 non-null  object        
 2   SOURCE_KEY       71808 non-null  object        
 3   DC_POWER         71808 non-null  float64       
 4   DAILY_YIELD      71808 non-null  float64       
 5   AMB_TEMP         71808 non-null  float64       
 6   MOD_TEMP         71808 non-null  float64       
 7   IRRADIATION      71808 non-null  float64       
 8   DATE             71808 non-null  object        
 9   TIME_OF_DAY      71808 non-null  object        
 10  HOUR             71808 non-null  int64         
 11  DAY              71808 non-null  int64         
 12  WEEKDAY          71808 non-null  object        
 13  MONTH            71808 non-null  int64         
 14  YEAR             71808 non-null  int64

In [6]:
# Drop unrequired data columns

# Identify columns to drop
cols_to_keep = ['DC_POWER', 'AMB_TEMP', 'MOD_TEMP', 'IRRADIATION', 'TIME_FLOAT']
cols_to_drop = []

for col in df_plant1.columns:
    if col not in cols_to_keep:
        cols_to_drop.append(col)

for df in [df_plant1, df_plant2]:
    df.drop(cols_to_drop, axis = 1, inplace = True)

df_plant2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71808 entries, 0 to 71807
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DC_POWER     71808 non-null  float64
 1   AMB_TEMP     71808 non-null  float64
 2   MOD_TEMP     71808 non-null  float64
 3   IRRADIATION  71808 non-null  float64
dtypes: float64(4)
memory usage: 2.7 MB


In [7]:
# Split data into training / evaluation sets
training_set_plant1, evaluation_set_plant1 = split_data.split_train_eval(df_plant1, 0.2)
training_set_plant2, evaluation_set_plant2 = split_data.split_train_eval(df_plant2, 0.2)

Original Data Items: 71808
Training Data Items: 57447
Evaluation Data Items: 14361
Original Data Items: 71808
Training Data Items: 57447
Evaluation Data Items: 14361


In [8]:
# Create DC Power Target datasets

# Modify training sets
dc_power_training_data_pt1 = training_set_plant1.drop('DC_POWER', axis = 1)
dc_label_data_pt1 = training_set_plant1['DC_POWER'].copy()

dc_power_training_data_pt2 = training_set_plant2.drop('DC_POWER', axis = 1)
dc_label_data_pt2 = training_set_plant2['DC_POWER'].copy()

# Modify evaluation sets
dc_evaluation_data_pt1 = evaluation_set_plant1.drop('DC_POWER', axis = 1)
dc_eval_label_data_pt1 = evaluation_set_plant1['DC_POWER'].copy()

dc_evaluation_data_pt2 = evaluation_set_plant2.drop('DC_POWER', axis = 1)
dc_eval_label_data_pt2 = evaluation_set_plant2['DC_POWER'].copy()

In [9]:
# Check shape of arrays
print('Plant 1:')
print('Fit Train:', dc_power_training_data_pt1.shape)
print('Fit Label:', dc_label_data_pt1.shape)
print('Eval Train:', dc_evaluation_data_pt1.shape)
print('Eval Label:', dc_eval_label_data_pt1.shape)

print('Plant 2:')
print('Fit Train:', dc_power_training_data_pt2.shape)
print('Fit Label:', dc_label_data_pt2.shape)
print('Eval Train:', dc_evaluation_data_pt2.shape)
print('Eval Label:', dc_eval_label_data_pt2.shape)

Plant 1:
Fit Train: (57447, 3)
Fit Label: (57447,)
Eval Train: (14361, 3)
Eval Label: (14361,)
Plant 2:
Fit Train: (57447, 3)
Fit Label: (57447,)
Eval Train: (14361, 3)
Eval Label: (14361,)


In [10]:
# Normalise Inputs
normalise_inputs = True
normalise_outputs = False

normaliser = MinMaxScaler()

if normalise_inputs == True:

    # Transform training sets
    dc_power_training_data_pt1 = normaliser.fit_transform(dc_power_training_data_pt1)
    dc_power_training_data_pt2 = normaliser.fit_transform(dc_power_training_data_pt2)

    # Transform evaluation sets
    dc_evaluation_data_pt1 = normaliser.fit_transform(dc_evaluation_data_pt1)
    dc_evaluation_data_pt2 = normaliser.fit_transform(dc_evaluation_data_pt2)

if normalise_outputs == True:

    # Transform training sets
    dc_label_data_pt1 = normaliser.fit_transform(dc_label_data_pt1)
    dc_label_data_pt2 = normaliser.fit_transform(dc_label_data_pt2)

    # Transform evaluation sets
    dc_eval_label_data_pt1 = normaliser.fit_transform(dc_eval_label_data_pt1)
    dc_eval_label_data_pt2 = normaliser.fit_transform(dc_eval_label_data_pt2)

In [11]:
# Check values after normalisation
dc_power_training_data_pt1

array([[0.69418927, 0.39836143, 0.24685468],
       [0.10297219, 0.06348367, 0.02010156],
       [0.10195445, 0.0510256 , 0.        ],
       ...,
       [0.3994604 , 0.20367104, 0.07955333],
       [0.31878711, 0.09578908, 0.        ],
       [0.45269604, 0.48094315, 0.44556097]])

In [12]:
# Create train and test arrays (plant 1)
X_train_pt1, X_test_pt1, y_train_pt1, y_test_pt1 = train_test_split(dc_power_training_data_pt1, dc_label_data_pt1, test_size = 0.2, random_state = 0)

print(X_train_pt1)
print(y_train_pt1)

[[0.77310878 0.77031963 0.62255649]
 [0.31338184 0.08745068 0.        ]
 [0.26876718 0.23906034 0.09710624]
 ...
 [0.11465821 0.06056427 0.        ]
 [0.15250767 0.05541926 0.        ]
 [0.25835535 0.07128375 0.        ]]
16030    936.842857
4052       0.000000
71148    178.350000
71233    154.514286
24592    414.100000
            ...    
14020    560.000000
50198     31.471429
65767      0.000000
50797      0.000000
12524      0.000000
Name: DC_POWER, Length: 45957, dtype: float64


In [13]:
# Initial MLP (Target - DC Power, Plant 1)

if TRAIN_MODELS == True:
    # Clear existing models
    keras.backend.clear_session()

    # Build model
    model = keras_helpers.build_multilayer_perceptron()

    # Name model
    model_type = "MLP_DC_Plant1"
    model_id = keras_helpers.name_model(model_type)
    filepath_full = keras_helpers.make_save_string(model_id)

    # Set save and earlystop callbacks
    earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

    # Train model
    model.fit(X_train_pt1, y_train_pt1, epochs = 20, validation_data = (X_test_pt1, y_test_pt1), callbacks =[checkpoint_cb, earlystop_cb])

Building Model ...
Hidden Layers: 2, Neurons: 6, LR: 0.001
Epoch 1/20
1437/1437 [==============================] - 3s 2ms/step - loss: 253079.2749 - val_loss: 230518.6406
Epoch 2/20
1437/1437 [==============================] - 2s 1ms/step - loss: 214021.5542 - val_loss: 142003.7969
Epoch 3/20
1437/1437 [==============================] - 4s 3ms/step - loss: 119865.3620 - val_loss: 74890.5547
Epoch 4/20
1437/1437 [==============================] - 2s 2ms/step - loss: 67270.2282 - val_loss: 46768.8555
Epoch 5/20
1437/1437 [==============================] - 2s 1ms/step - loss: 39692.7710 - val_loss: 21804.9648
Epoch 6/20
1437/1437 [==============================] - 2s 1ms/step - loss: 16685.9653 - val_loss: 9288.6934
Epoch 7/20
1437/1437 [==============================] - 2s 1ms/step - loss: 8286.8625 - val_loss: 7260.3906
Epoch 8/20
1437/1437 [==============================] - 2s 1ms/step - loss: 6741.0564 - val_loss: 6304.8208
Epoch 9/20
1437/1437 [==============================] - 2s 1m

In [14]:
# Evaluate MLP Model (DC Power, Plant 1)

if TRAIN_MODELS == False:
    filepath_full = r'C:\Developer\solar-power-generation-project\Models\WJ_MLP_DC_2021_01_13-16_10_50.h5'

# Load model
model = keras.models.load_model(filepath_full)

# Summarise model
model.summary()

# Make predictions
dc_pred_eval = model.predict(dc_evaluation_data_pt1)

# Determine model prediction stats
model_name = "MLP_DC_Plant1"
model_evaluation.evaluate_model(model_name, dc_eval_label_data_pt1, dc_pred_eval)
rmse, mae, r2 = model_evaluation.return_model_evaluation_stats(dc_eval_label_data_pt1, dc_pred_eval)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 24        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 73
Trainable params: 73
Non-trainable params: 0
_________________________________________________________________
MLP_DC_Plant1 rmse (Eval): 60.852316475907756
MLP_DC_Plant1 mae (Eval): 27.97568432308068
MLP_DC_Plant1 r2 (Eval): 0.9767674629339826


In [15]:
# Setup tensorboard for logging 
x = tensorboard_helpers.get_run_logdir()

# Print tensorboard directory
print(x)

c:\Developer\solar-power-generation-project\Models\TensorBoard\run_2021_06_10-15_38_34


In [16]:
# Optimised MLP (Target - DC Power, Plant 1)

if TRAIN_MODELS == True:
    
    # Clear existing models
    keras.backend.clear_session()

    # Establish parameter distribution for tuning
    param_distribs = {
        "n_hidden": (4, 8, 12, 16, 20),
        "n_neurons": (5, 10, 20, 40, 60, 80, 100),
        "learning_rate": (1e-1, 1e-2, 1e-3, 1e-4),
    }

    # Build model
    wrapped_model = keras_helpers.wrap_model()

    # Initialise random search
    rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

    # Name model
    model_type = "MLP_Opt_DC_Plant1"
    model_id = keras_helpers.name_model(model_type)
    filepath_full = keras_helpers.make_save_string(model_id)

    # Set save and earlystop callbacks
    earlystop_cb = keras.callbacks.EarlyStopping(patience = 3)
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

    # Set TensorBoard callback for logging
    tb_logdir = tensorboard_helpers.get_run_logdir()
    tensorboard_cb = keras.callbacks.TensorBoard(tb_logdir)

    # Train model
    rnd_search_cv.fit(X_train_pt1, y_train_pt1, epochs = 100, validation_data = (X_test_pt1, y_test_pt1), callbacks = [checkpoint_cb, earlystop_cb, tensorboard_cb], verbose = 0)

Building Model ...
Hidden Layers: 8, Neurons: 20, LR: 0.001
479/479 [==============================] - 0s 809us/step - loss: 3497.5195
Building Model ...
Hidden Layers: 8, Neurons: 20, LR: 0.001
479/479 [==============================] - 0s 784us/step - loss: 3112.4565
Building Model ...
Hidden Layers: 8, Neurons: 20, LR: 0.001
479/479 [==============================] - 0s 783us/step - loss: 3362.1572
Building Model ...
Hidden Layers: 12, Neurons: 80, LR: 0.1
479/479 [==============================] - 0s 867us/step - loss: nan
Building Model ...
Hidden Layers: 12, Neurons: 80, LR: 0.1
479/479 [==============================] - 0s 887us/step - loss: 162480.5938
Building Model ...
Hidden Layers: 12, Neurons: 80, LR: 0.1
479/479 [==============================] - 0s 907us/step - loss: nan
Building Model ...
Hidden Layers: 4, Neurons: 40, LR: 0.1
479/479 [==============================] - 0s 801us/step - loss: 6303.4365
Building Model ...
Hidden Layers: 4, Neurons: 40, LR: 0.1
479/479 [===

In [17]:
# Evaluate MLP Model (DC Power, Plant 1)

if TRAIN_MODELS == False:
    filepath_full = r'C:\Developer\solar-power-generation-project\Models\WJ_MLP_Opt_DC_Plant1_2021_06_10-11_34_55.h5'

wrapped_model = keras.models.load_model(filepath_full)

# Summarise model
wrapped_model.summary()

# Make predictions
dc_pred_eval = wrapped_model.predict(dc_evaluation_data_pt1)

model_name = "MLP_Opt_DC_pt1"
model_evaluation.evaluate_model(model_name, dc_eval_label_data_pt1, dc_pred_eval)
rmse, mae, r2 = model_evaluation.return_model_evaluation_stats(dc_eval_label_data_pt1, dc_pred_eval)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_285 (Dense)            (None, 100)               400       
_________________________________________________________________
dense_286 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_287 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_288 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_289 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_290 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_291 (Dense)            (None, 100)             

In [18]:
# Create train and test arrays (plant 2)
X_train_pt2, X_test_pt2, y_train_pt2, y_test_pt2 = train_test_split(dc_power_training_data_pt2, dc_label_data_pt2, test_size = 0.2, random_state = 0)

print(X_train_pt2)
print(y_train_pt2)

[[0.78718932 0.86385524 0.75714748]
 [0.40111725 0.13940226 0.        ]
 [0.366359   0.24734289 0.16542722]
 ...
 [0.17626336 0.06610678 0.        ]
 [0.19348786 0.04786465 0.        ]
 [0.30564703 0.10208519 0.        ]]
16031    1196.980000
4052        0.000000
71148     285.164286
71234     140.678571
24593     896.435714
            ...     
14020     872.093333
50199      57.906667
65768       0.000000
50783       0.000000
12524       0.000000
Name: DC_POWER, Length: 45957, dtype: float64


In [19]:
# Initial MLP (Target - DC Power, Plant 2)

if TRAIN_MODELS == True:
    
    # Clear existing models
    keras.backend.clear_session()

    # Build model
    model = keras_helpers.build_multilayer_perceptron()

    # Name model
    model_type = "MLP_DC_Plant2"
    model_id = keras_helpers.name_model(model_type)
    filepath_full = keras_helpers.make_save_string(model_id)

    # Set save and earlystop callbacks
    earlystop_cb = keras.callbacks.EarlyStopping(patience = 5)
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

    # Train model
    model.fit(X_train_pt2, y_train_pt2, epochs = 20, validation_data = (X_test_pt2, y_test_pt2), callbacks =[checkpoint_cb, earlystop_cb])

Building Model ...
Hidden Layers: 2, Neurons: 6, LR: 0.001
Epoch 1/20
1437/1437 [==============================] - 2s 1ms/step - loss: 199260.8374 - val_loss: 188264.5312
Epoch 2/20
1437/1437 [==============================] - 2s 1ms/step - loss: 179267.1196 - val_loss: 139403.8750
Epoch 3/20
1437/1437 [==============================] - 2s 1ms/step - loss: 120203.3654 - val_loss: 77343.1016
Epoch 4/20
1437/1437 [==============================] - 2s 1ms/step - loss: 68039.2063 - val_loss: 53710.2930
Epoch 5/20
1437/1437 [==============================] - 2s 1ms/step - loss: 51220.3296 - val_loss: 50347.7266
Epoch 6/20
1437/1437 [==============================] - 2s 1ms/step - loss: 49203.5143 - val_loss: 49140.3711
Epoch 7/20
1437/1437 [==============================] - 2s 1ms/step - loss: 48315.5818 - val_loss: 48765.1250
Epoch 8/20
1437/1437 [==============================] - 2s 1ms/step - loss: 46926.3703 - val_loss: 48611.1016
Epoch 9/20
1437/1437 [==============================] - 

In [20]:
# Evaluate MLP Model (DC Power, Plant 2)

if TRAIN_MODELS == False:
    filepath_full = r'C:\Developer\solar-power-generation-project\Models\WJ_MLP_DC_2021_01_13-16_10_50.h5'

# Load model
model = keras.models.load_model(filepath_full)

# Summarise model
model.summary()

# Make predictions
dc_pred_eval = model.predict(dc_evaluation_data_pt2)

# Determine model prediction stats
model_name = "MLP_DC"
model_evaluation.evaluate_model(model_name, dc_eval_label_data_pt2, dc_pred_eval)
rmse, mae, r2 = model_evaluation.return_model_evaluation_stats(dc_eval_label_data_pt2, dc_pred_eval)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 24        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 73
Trainable params: 73
Non-trainable params: 0
_________________________________________________________________
MLP_DC rmse (Eval): 220.21751981349564
MLP_DC mae (Eval): 100.6863230268892
MLP_DC r2 (Eval): 0.6487816689527413


In [21]:
# Setup tensorboard for logging 
x = tensorboard_helpers.get_run_logdir()

# Print tensorboard directory
print(x)

c:\Developer\solar-power-generation-project\Models\TensorBoard\run_2021_06_10-15_58_20


In [22]:
# Optimised MLP (Target - DC Power, Plant 2)

if TRAIN_MODELS == True:
    
    # Clear existing models
    keras.backend.clear_session()

    # Establish parameter distribution for tuning
    param_distribs = {
        "n_hidden": (4, 8, 12, 16),
        "n_neurons": (5, 10, 20, 40, 60, 80, 100),
        "learning_rate": (1e-1, 1e-2, 1e-3, 1e-4),
    }

    # Build model
    wrapped_model = keras_helpers.wrap_model()

    # Initialise random search
    rnd_search_cv = RandomizedSearchCV(wrapped_model, param_distribs, n_iter = 10, cv = 3)

    # Name model
    model_type = "MLP_Opt_DC_Plant1"
    model_id = keras_helpers.name_model(model_type)
    filepath_full = keras_helpers.make_save_string(model_id)

    # Set save and earlystop callbacks
    earlystop_cb = keras.callbacks.EarlyStopping(patience = 3)
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = filepath_full, save_best_only = True)

    # Set TensorBoard callback for logging
    tb_logdir = tensorboard_helpers.get_run_logdir()
    tensorboard_cb = keras.callbacks.TensorBoard(tb_logdir)

    # Train model
    rnd_search_cv.fit(X_train_pt2, y_train_pt2, epochs = 100, validation_data = (X_test_pt2, y_test_pt2), callbacks = [checkpoint_cb, earlystop_cb, tensorboard_cb], verbose = 0)

Building Model ...
Hidden Layers: 8, Neurons: 80, LR: 0.1
479/479 [==============================] - 0s 867us/step - loss: 51110.5547
Building Model ...
Hidden Layers: 8, Neurons: 80, LR: 0.1
479/479 [==============================] - 0s 864us/step - loss: 137943.2656
Building Model ...
Hidden Layers: 8, Neurons: 80, LR: 0.1
479/479 [==============================] - 0s 873us/step - loss: 137857.8438
Building Model ...
Hidden Layers: 4, Neurons: 60, LR: 0.0001
479/479 [==============================] - 0s 773us/step - loss: 46360.1797
Building Model ...
Hidden Layers: 4, Neurons: 60, LR: 0.0001
479/479 [==============================] - 0s 790us/step - loss: 45711.6406
Building Model ...
Hidden Layers: 4, Neurons: 60, LR: 0.0001
479/479 [==============================] - 0s 791us/step - loss: 48241.8047
Building Model ...
Hidden Layers: 4, Neurons: 20, LR: 0.01
479/479 [==============================] - 0s 763us/step - loss: 48049.1328
Building Model ...
Hidden Layers: 4, Neurons: 20, 

In [24]:
# Evaluate MLP Model (DC Power, Plant 2)

if TRAIN_MODELS == False:
    filepath_full = r'C:\Developer\solar-power-generation-project\Models\WJ_MLP_Opt_DC_2021_01_13-15_30_27.h5'

wrapped_model = keras.models.load_model(filepath_full)

# Summarise model
wrapped_model.summary()

# Make predictions
dc_pred_eval = wrapped_model.predict(dc_evaluation_data_pt2)

model_name = "MLP_Opt_DC_pt2"
model_evaluation.evaluate_model(model_name, dc_eval_label_data_pt2, dc_pred_eval)
rmse, mae, r2 = model_evaluation.return_model_evaluation_stats(dc_eval_label_data_pt2, dc_pred_eval)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_201 (Dense)            (None, 100)               400       
_________________________________________________________________
dense_202 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_203 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_204 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_205 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_206 (Dense)            (None, 100)               10100     
_________________________________________________________________
dense_207 (Dense)            (None, 100)             